# Given an initial investment amount, this code can be used to determine the combination of stocks that will yield the highest return based on dividend yield history.

In [29]:
import pandas as pd
import numpy as np
import plotly 
import yfinance as yf
import heapq
from datetime import datetime

In [31]:

sp500_tickers = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]['Symbol'].tolist()

# Define desired monthly dividend income
# Loop over each ticker and analyze its dividend 
for ticker in sp500_tickers:
    ticker = yf.Ticker(ticker)
    dividends = ticker.dividends

    # Calculate average monthly dividend income 
    avg_monthly_dividend = dividends.resample("M").sum().mean()
    monthly_income = 1000
    # Determine if stock meets your income goal 
    if avg_monthly_dividend >= monthly_income:
        print(f"{ticker} meets desired monthly dividend income with an average monthly dividend of {avg_monthly_dividend:.2f}")
    else:
        print(f"{ticker} does not meet desired monthly dividend income with an average monthly dividend of {avg_monthly_dividend:.2f}")
    break


yfinance.Ticker object <MMM> does not meet desired monthly dividend income with an average monthly dividend of 0.12


In [36]:
import yfinance as yf

def get_top_dividend_stocks(investment_amount, n_stocks=10):
    #tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'T', 'VZ', 'IBM', 'XOM', 'JNJ', 'PFE']
    tickers = sp500_tickers
    dividend_data = []

    for ticker in tickers:
        stock_info = yf.Ticker(ticker)
        div_yield = stock_info.info.get('dividendYield', 0)
        dividend_data.append((ticker, div_yield))

    top_dividend_stocks = heapq.nlargest(n_stocks, dividend_data, key=lambda x: x[1])

    total_dividend_yield = sum([stock[1] for stock in top_dividend_stocks])
    allocations = [(stock[0], stock[1] / total_dividend_yield * investment_amount) for stock in top_dividend_stocks]

    total_dividends = 0
    for stock, allocation in allocations:
        stock_info = yf.Ticker(stock)
        dividend_yield = stock_info.info.get('dividendYield', 0)
        total_dividends += allocation * dividend_yield

    return allocations, total_dividends

investment_amount = 10000  # Invest $10,000
top_dividend_stocks, total_dividends = get_top_dividend_stocks(investment_amount)

print("Investment allocations:")
for stock, allocation in top_dividend_stocks:
    print(f"{stock}: ${allocation:.2f}")

print(f"\nTotal dividends for the year: ${total_dividends:.2f}")


Investment allocations:
PXD: $1460.09
DVN: $1141.99
LNC: $999.39
MO: $996.95
CTRA: $977.45
FANG: $953.08
BXP: $914.08
NWL: $894.58
VZ: $855.58
KEY: $806.83

Total dividends for the year: $846.04
